In [22]:
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
print(train['Embarked'].value_counts())

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


1. passenger id와 name은 생존여부와는 너무 무관하니 제외
2. Cabin은 결측치가 너무 많기 때문에 활용하기 어려워 보임
3. Ticket은 형식이 통일되어 있지 않을 뿐 더러, ticket number가 생존 여부를 갈랐다고 보기 어려움
4. Pclass, Sex, Age, SibSp, Parch, Fare, Embarked 를 활용.
5. 결측값 대치, str열 대치
6. 히스토그램 기반 그레이디언트 부스팅을 활용해 훈련하고 ```permutation_importance()```를 통해 중요도가 낮은 특성 칼럼은 제거

In [6]:
train_input = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
train_target = train['Survived']

In [23]:
train_input['Age'] = train_input['Age'].fillna(train_input['Age'].mean())               
train_input['Embarked'] = train_input['Embarked'].fillna(train_input['Embarked'].mode())
#train_input의 결측값 대치

test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
#test의 결측값 대치

train_input['Embarked'] = train_input['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})
test['Embarked'] = test['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})

train_input['Sex'] = train_input['Sex'].replace({'male': 1, 'female': 2})
test['Sex'] = test['Sex'].replace({'male': 1, 'female': 2})
#성별 str을 numeric으로 대치

C:\Users\1-20\AppData\Local\Temp\ipykernel_18044\2410624915.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_input['Embarked'] = train_input['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})
C:\Users\1-20\AppData\Local\Temp\ipykernel_18044\2410624915.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Embarked'] = test['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2})
C:\Users\1-20\AppData\Local\Temp\ipykernel_18044\2410624915.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a fu

In [24]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = \
train_test_split(train_input, train_target)
#검증세트 분할

In [25]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

hgb = HistGradientBoostingClassifier()
scores = cross_validate(
    hgb, train_input, train_target,
    return_train_score=True, n_jobs=-1)

print(scores)

{'fit_time': array([0.07998705, 0.04930139, 0.10833645, 0.0854423 , 0.09412909]), 'score_time': array([0.00656557, 0.00270629, 0.00302696, 0.00400281, 0.00363541]), 'test_score': array([0.76      , 0.76      , 0.77333333, 0.76      , 0.84      ]), 'train_score': array([0.92333333, 0.95      , 0.94666667, 0.92666667, 0.93      ])}


-여기까지 과대적합된 결과가 나왔으므로 중요도 체크 해서 특성 몇 개 날리고 그리드 서치로 파라미터 조정 들어가자